In [1]:
!pip install pandas numpy scikit-learn nltk matplotlib seaborn



Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 473.3 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 473.3 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 473.3 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 473.3 kB/s eta 0:00:03
   -------------------- ------------------- 0.8/1.5 MB 377.7 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [3]:
# load dataset
data = pd.read_csv("train_data.txt", sep=":::", engine="python",
                   names=["id", "title", "genre", "description"])

data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'train_data.txt'

In [1]:
import os
os.getcwd()


'C:\\Users\\Admin\\Desktop\\codsoft movie genre'

In [2]:
import pandas as pd

# load training dataset
data = pd.read_csv(
    "train_data.txt",
    sep=":::", 
    engine="python",
    names=["id", "title", "genre", "description"],
    encoding="latin-1"
)

data.head()


,id,title,genre,description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [3]:
data.shape


(54214, 4)

In [4]:
"Listening in to a conversation between his daughter..."


'Listening in to a conversation between his daughter...'

In [5]:
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()                          # lowercase
    text = re.sub(r'[^a-zA-Z]', ' ', text)       # remove numbers & symbols
    text = text.split()                          # tokenize words
    text = [word for word in text if word not in stop_words]  # remove stopwords
    text = ' '.join(text)
    return text

data['clean_description'] = data['description'].apply(clean_text)

data.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


,id,title,genre,description,clean_description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...,listening conversation doctor parents year old...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous r...,brother sister past incestuous relationship cu...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...,bus empties students field trip museum natural...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...,help unemployed father make ends meet edith tw...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...,film title refers un recovered bodies ground z...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)

X = tfidf.fit_transform(data['clean_description']).toarray()
y = data['genre']

print("X shape:", X.shape)


X shape: (54214, 5000)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)


,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [9]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5216268560361523


In [10]:
def predict_genre(text):

    # clean the text (same cleaning we used before)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = ' '.join(text)

    # convert text to vector
    vector = tfidf.transform([text]).toarray()

    # predict
    prediction = model.predict(vector)

    return prediction[0]


In [11]:
predict_genre("A young warrior fights against evil forces to save his kingdom and rescue the princess.")


np.str_(' action ')

In [12]:
predict_genre("Two college students slowly fall in love but their families are enemies and try to separate them.")


np.str_(' drama ')

In [13]:
predict_genre("A group of friends enter an abandoned house and a ghost starts killing them one by one at night.")


np.str_(' horror ')